# Data Analysis and GIS Storytelling

In this notebook we will explore 311 + ACS Data.

_Note: This is a starter / placeholder file for Felix / Leslie to work on._

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import seaborn as sns
import pydeck as pdk

In [ ]:
# Load 311 service requests and ACS data
serv_req_gdf = gpd.read_parquet('data/processed/serv_req_cleaned.parquet')
sf_tracts_acs = gpd.read_file('data/processed/sf_tracts_cleaned.gpkg')

print("311 Service Requests GeoDataFrame:")
display(serv_req_gdf.head())
print("San Francisco Tracts ACS GeoDataFrame:")
display(sf_tracts_acs.head())